![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de popularidad en canción

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de popularidad en canción".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad-2025-12-prediccion-popularidad-en-cancion).

## Datos para la predicción de popularidad en cancion

En este proyecto se usará el conjunto de datos de datos de popularidad en canciones, donde cada observación representa una canción y se tienen variables como: duración de la canción, acusticidad y tempo, entre otras. El objetivo es predecir qué tan popular es la canción. Para más detalles puede visitar el siguiente enlace: [datos](https://huggingface.co/datasets/maharshipandya/spotify-tracks-dataset).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
# Importación de Librerías
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2025/main/datasets/dataTrain_Spotify.csv')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2025/main/datasets/dataTest_Spotify.csv', index_col=0)

In [ ]:
# Visualización datos de entrenamiento
dataTraining.head()

In [ ]:
# Visualización datos de test
dataTesting.head()

## Preprocesamiento de Datos

### 1. Revisión inicial de los datos
En esta sección se realiza una revisión inicial de los datos en la que se extrae la dimensión de los datos, se verifica la existencia de valores nulos, duplicados y el tipo de datos que hay en cada variable.

In [ ]:
# Revisión inicial de datos
print("Dimensión de los datos de entrenamiento:", dataTraining.shape)

if dataTraining.isnull().sum().sum() > 0: # Verifica si hay valores nulos
    print("\nValores nulos en los datos de entrenamiento:")
    print(dataTraining.isnull().sum())
else:
    print("\nNo hay valores nulos en los datos de entrenamiento.")

print("\nNúmero de canciones duplicadas según Track_ID:")
print(dataTraining['track_id'].duplicated().sum())

print("\nTipos de variables en la base de datos:")
print(dataTraining.dtypes.unique())

print("\nVariables categóricas en la base de datos:")
print(dataTraining.select_dtypes(include=['object']).info())

### 2. Limpieza de la base de datos
En esta sección se realiza una depuración de los valores duplicados indentificados en el punto anterior, se depuran algunas variables que no serán incluidas en el modelo y se convierten a enteros las variables booleanas que fueron identificadas en la exploración inicial.

In [ ]:
# Limpieza de la base de datos
data = dataTraining.drop_duplicates(subset='track_id', keep='first') # Eliminar duplicados
data = data.drop(columns=['Unnamed: 0','track_id', 'artists', 'album_name', 'track_name', 'track_genre']) # Excluir columnas string

columns_factorize = [col for col in data.columns if data[col].dtype == 'bool']
for col in columns_factorize:
    data[col] = pd.factorize(data[col])[0] # Convertir variables booleanas a enteros

### 3. Separación de datos de entrenamiento y prueba
En esta sección se realiza la separación de los datos en entrenamiento y prueba según una partición de 67% - 33%.

In [ ]:
# Separación de variables predictoras (X) y variable de interés (y)
X = data.drop(columns=['popularity']) # Variables predictoras
y = data['popularity'] # Variable de interés

# División de los datos en conjuntos de entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 

### 4. Análisis descriptivo

In [ ]:
# Estadísticas descriptivas de los datos de entrenamiento
print("\nEstadísticas descriptivas de los datos de entrenamiento:")
X_train.describe()
print("\nEstadísticas descriptivas de la variable de interés:")
print(y_train.describe())

## Calibración y Entrenamiento del Modelo Predictivo

In [ ]:
# Definicion y entrenamiento del modelo XGBRegressor
XGBReg = XGBRegressor()

# Entrenamiento (fit) y desempeño del modelo XGBRegressor
XGBReg.fit(X_train, y_train)
pred_XGBReg = XGBReg.predict(X_test)

In [ ]:
# Evaluación del modelo
MAE_XGBReg = mean_absolute_error(y_test, pred_XGBReg)
RMSE_XGBReg = np.sqrt(mean_squared_error(y_test, pred_XGBReg))
R2_XGBReg = r2_score(y_test, pred_XGBReg)

# Resultados
print('MAE: ', f'{MAE_XGBReg:.4f}')
print('RMSE: ', f'{RMSE_XGBReg:.4f}')
print('R2: ', f'{R2_XGBReg:.4f}')

MAE:  15.4310
RMSE:  19.3890
R2:  0.1482


## Generación Predicción Kaggle

In [ ]:
y_pred = XGBReg.predict(X_test)  # predicción sobre el conjunto de test
# Convertir y_pred a DataFrame
y_pred_df = pd.DataFrame(y_pred, index=dataTesting.index, columns=['Popularity'])
# Guardar en CSV
y_pred_df.to_csv('test_submission_xgbreg.csv', index_label='ID')
y_pred_df.head()